Using the Sakila database I am starting by setting up my console. This involves making sure that SQL Alchemy and PyMongo are working accordingly. 

In [29]:
# import libraries
import os
import json
import pandas as pd
from sqlalchemy import create_engine, text 
from pymongo import MongoClient 
import sqlalchemy 
import pymongo 

# MySQL connection
mysql_user = 'root'
mysql_password = 'Kaytie#2003' 
mysql_host = 'localhost'
mysql_db = 'sakila_project'

# MongoDB connection, I used the mongodb that is connected to my desktop instead of my browser
mongo_client = MongoClient("mongodb://localhost:27017/")
mongo_db = mongo_client["sakila_project"]

# Connect to MySQL and create the project database
sakila_project_engine = create_engine(f"mysql+mysqlconnector://{mysql_user}:{mysql_password}@{mysql_host}/{mysql_db}")
with sakila_project_engine.connect() as conn:
    conn.execute(text("DROP DATABASE IF EXISTS sakila_project"))
    conn.execute(text("CREATE DATABASE sakila_project"))
    conn.execute(text("USE sakila_project"))

print("Database 'sakila_project' created successfully.")


Database 'sakila_project' created successfully.


In [30]:
print(f"Running SQLAlchemy Version: {sqlalchemy.__version__}")
print(f"Running PyMongo Version: {pymongo.version}")

Running SQLAlchemy Version: 2.0.30
Running PyMongo Version: 3.11.4


In [31]:
# Paths to JSON files
json_files = {
    "customer_collection": "C:\\Users\\kayti\\OneDrive\\Data Science Systems\\customer.json",
    "film_collection": "C:\\Users\\kayti\\OneDrive\\Data Science Systems\\film.json",  
    "inventory_collection": "C:\\Users\\kayti\\OneDrive\\Data Science Systems\\inventory.json" 
}

# Function to load JSON data into MongoDB
def load_json_to_mongo(collection_name, file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    collection = mongo_db[collection_name]
    collection.insert_many(data)
    # making sure it accurately uploaded 
    print(f"Data loaded into MongoDB collection '{collection_name}'.")

# Load each JSON file into MongoDB
for collection_name, file_path in json_files.items():
    load_json_to_mongo(collection_name, file_path)

mongo_client.close()
print("All JSON data has been loaded into MongoDB successfully.")



Data loaded into MongoDB collection 'customer_collection'.
Data loaded into MongoDB collection 'film_collection'.
Data loaded into MongoDB collection 'inventory_collection'.
All JSON data has been loaded into MongoDB successfully.


Create dim_customer table

In [32]:
# creating dim_customer table
create_customer_table_query = text("""
CREATE TABLE IF NOT EXISTS dim_customer (
    customer_id INT PRIMARY KEY,
    store_id INT,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(100),
    address_id INT,
    active TINYINT,
    create_date DATETIME,
    last_update DATETIME
);
""")

with sakila_project_engine.connect() as conn:
    conn.execute(create_customer_table_query)
    print("dim_customer table created successfully.")



dim_customer table created successfully.


In [33]:
# populate dim_customer
customer_data = pd.DataFrame(list(mongo_db["customer_collection"].find()))
customer_data = customer_data.drop(columns=['_id'], errors='ignore')
customer_data.to_sql('dim_customer', con=sakila_project_engine, if_exists='replace', index=False)



13178

Run Lab 2c to create dim_date now

In [34]:
# Verify dim_date in Jupyter Notebook
date_data_check = pd.read_sql("SELECT * FROM dim_date LIMIT 5;", con=sakila_project_engine)
print("dim_date Data Check:")
date_data_check.head(5)


dim_date Data Check:


,date_key,full_date,date_name,date_name_us,date_name_eu,day_of_week,day_name_of_week,day_of_month,day_of_year,weekday_weekend,...,is_last_day_of_month,calendar_quarter,calendar_year,calendar_year_month,calendar_year_qtr,fiscal_month_of_year,fiscal_quarter,fiscal_year,fiscal_year_month,fiscal_year_qtr
0,20000101,2000-01-01,2000/01/01,01/01/2000,01/01/2000,7,Saturday,1,1,Weekend,...,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3
1,20000102,2000-01-02,2000/01/02,01/02/2000,02/01/2000,1,Sunday,2,2,Weekend,...,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3
2,20000103,2000-01-03,2000/01/03,01/03/2000,03/01/2000,2,Monday,3,3,Weekday,...,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3
3,20000104,2000-01-04,2000/01/04,01/04/2000,04/01/2000,3,Tuesday,4,4,Weekday,...,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3
4,20000105,2000-01-05,2000/01/05,01/05/2000,05/01/2000,4,Wednesday,5,5,Weekday,...,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3


Create Fact Table and inert data

In [35]:
create_fact_table_query = text("""
CREATE TABLE IF NOT EXISTS fact_rental (
    rental_id INT PRIMARY KEY,
    rental_date DATETIME,
    return_date DATETIME,
    customer_id INT,
    film_id INT,
    inventory_id INT,
    staff_id INT
);
""")

with sakila_project_engine.connect() as conn:
    conn.execute(create_fact_table_query)
    print("fact_rental table created successfully.")


fact_rental table created successfully.


In [36]:
insert_data_query = text("""
INSERT IGNORE INTO fact_rental (rental_id, rental_date, return_date, customer_id, film_id, inventory_id, staff_id)
SELECT r.rental_id, r.rental_date, r.return_date, r.customer_id, i.film_id, r.inventory_id, r.staff_id
FROM sakila.rental r
JOIN sakila.inventory i ON r.inventory_id = i.inventory_id
JOIN sakila.film f ON i.film_id = f.film_id
JOIN sakila.customer c ON r.customer_id = c.customer_id
JOIN sakila.staff s ON r.staff_id = s.staff_id
""")

with sakila_project_engine.connect() as conn:
    conn.execute(insert_data_query)
    conn.commit()  # Explicitly commit the transaction
    print("Data successfully inserted into fact_rental")


Data successfully inserted into fact_rental


I ran into many issues involving duplicates in the Sakila database, so I had to remove duplicates.

In [37]:
# Check for duplicate rental_id values
duplicate_check = pd.read_sql("""
SELECT rental_id, COUNT(*)
FROM sakila.rental
GROUP BY rental_id
HAVING COUNT(*) > 1
""", con=sakila_project_engine)

print(duplicate_check)


Empty DataFrame
Columns: [rental_id, COUNT(*)]
Index: []


Verify that data was loaded correctly into fact rental table

In [38]:
fact_rental_check = pd.read_sql("SELECT * FROM fact_rental LIMIT 5;", con=sakila_project_engine)
print("Fact Rental Table Data Check after full insertion:")
fact_rental_check.head(5)


Fact Rental Table Data Check after full insertion:


,rental_id,rental_date,return_date,customer_id,film_id,inventory_id,staff_id
0,1,2005-05-24 22:53:30,2005-05-26 22:04:30,130,80,367,1
1,2,2005-05-24 22:54:33,2005-05-28 19:40:33,459,333,1525,1
2,3,2005-05-24 23:03:39,2005-06-01 22:12:39,408,373,1711,1
3,4,2005-05-24 23:04:41,2005-06-03 01:43:41,333,535,2452,2
4,5,2005-05-24 23:05:21,2005-06-02 04:33:21,222,450,2079,1


Create Dim Film and insert data

In [39]:
create_dim_film_query = text("""
CREATE TABLE IF NOT EXISTS dim_film (
    rental_id INT PRIMARY KEY,
    rental_date DATETIME,
    return_date DATETIME,
    customer_id INT,
    film_id INT,
    inventory_id INT,
    staff_id INT
);
""")

with sakila_project_engine.connect() as conn:
    conn.execute(create_dim_film_query)
    print("dim_film table created successfully.")


dim_film table created successfully.


In [40]:
insert_data_query = text("""
INSERT IGNORE INTO dim_film (rental_id, rental_date, return_date, customer_id, film_id, inventory_id, staff_id)
SELECT r.rental_id, r.rental_date, r.return_date, r.customer_id, i.film_id, r.inventory_id, r.staff_id
FROM sakila.rental r
JOIN sakila.inventory i ON r.inventory_id = i.inventory_id
JOIN sakila.customer c ON r.customer_id = c.customer_id
JOIN sakila.staff s ON r.staff_id = s.staff_id;
""")

with sakila_project_engine.connect() as conn:
    conn.execute(insert_data_query)
    conn.commit()  # Commit the transaction
    print("Data successfully inserted into dim_film with commit.")


Data successfully inserted into dim_film with commit.


Check that all data was updated correctly into dim film

In [41]:
# Verify data in MySQL
film_check = pd.read_sql("SELECT * FROM dim_film LIMIT 10;", con=sakila_project_engine)
print("Sample data from dim_film table:")
film_check.head(10)


Sample data from dim_film table:


,rental_id,rental_date,return_date,customer_id,film_id,inventory_id,staff_id
0,1,2005-05-24 22:53:30,2005-05-26 22:04:30,130,80,367,1
1,2,2005-05-24 22:54:33,2005-05-28 19:40:33,459,333,1525,1
2,3,2005-05-24 23:03:39,2005-06-01 22:12:39,408,373,1711,1
3,4,2005-05-24 23:04:41,2005-06-03 01:43:41,333,535,2452,2
4,5,2005-05-24 23:05:21,2005-06-02 04:33:21,222,450,2079,1
5,6,2005-05-24 23:08:07,2005-05-27 01:32:07,549,613,2792,1
6,7,2005-05-24 23:11:53,2005-05-29 20:34:53,269,870,3995,2
7,8,2005-05-24 23:31:46,2005-05-27 23:33:46,239,510,2346,2
8,9,2005-05-25 00:00:40,2005-05-28 00:22:40,126,565,2580,1
9,10,2005-05-25 00:02:21,2005-05-31 22:44:21,399,396,1824,2


Create fact inventory availability and insert data

In [42]:
from sqlalchemy import text

# Define the SQL query to create the table
create_table_query = text("""
    CREATE TABLE IF NOT EXISTS fact_inventory_availability (
        availability_key INT PRIMARY KEY,
        date_key INT,
        film_id INT,
        store_key INT,
        inventory_count INT
    );
""")

# Execute the create table query within a connection
with sakila_project_engine.connect() as conn:
    conn.execute(create_table_query)
    conn.commit()  # Commit the transaction

print("fact_inventory_availability table created successfully.")


fact_inventory_availability table created successfully.


In [43]:
from sqlalchemy import text

insert_query = text("""
    INSERT INTO fact_inventory_availability (availability_key, date_key, film_id, store_key, inventory_count)
    VALUES
        (1, 20050501, 1, 1, 50),
        (2, 20050501, 2, 1, 30),
        (3, 20050501, 3, 2, 20),
        (4, 20050502, 4, 3, 10),
        (5, 20050503, 5, 4, 5);
""")

# Execute the insert query within a connection
with sakila_project_engine.connect() as conn:
    conn.execute(insert_query)
    conn.commit()  # Commit the transaction

print("Data inserted into fact_inventory_availability table successfully.")


Data inserted into fact_inventory_availability table successfully.


Ensure that data was uploaded correctly

In [44]:
# Check that it runs
fa_check = pd.read_sql("SELECT * FROM fact_inventory_availability LIMIT 10;", con=sakila_project_engine)
print("Sample data from fact_inventory_availability:")
fa_check.head(5)


Sample data from fact_inventory_availability:


,availability_key,date_key,film_id,store_key,inventory_count
0,1,20050501,1,1,50
1,2,20050501,2,1,30
2,3,20050501,3,2,20
3,4,20050502,4,3,10
4,5,20050503,5,4,5


Check that everything runs correctly

In [45]:
related_data_check = pd.read_sql("""
SELECT 
    fa.date_key,
    fa.film_id,  -- Corrected column name
    fa.store_key,
    fa.inventory_count,
    f.title
FROM fact_inventory_availability fa
JOIN dim_film df ON fa.film_id = df.film_id
JOIN sakila.film f ON df.film_id = f.film_id
LIMIT 10;
""", con=sakila_project_engine)

related_data_check.head(10)


,date_key,film_id,store_key,inventory_count,title
0,20050501,1,1,50,ACADEMY DINOSAUR
1,20050502,4,3,10,AFFAIR PREJUDICE
2,20050503,5,4,5,AFRICAN EGG
3,20050501,1,1,50,ACADEMY DINOSAUR
4,20050502,4,3,10,AFFAIR PREJUDICE
5,20050501,3,2,20,ADAPTATION HOLES
6,20050501,1,1,50,ACADEMY DINOSAUR
7,20050501,3,2,20,ADAPTATION HOLES
8,20050503,5,4,5,AFRICAN EGG
9,20050502,4,3,10,AFFAIR PREJUDICE


In [46]:
inventory_summary = pd.read_sql("""
SELECT 
    f.title,  -- Fetch title from the film table
    SUM(fa.inventory_count) AS total_inventory
FROM fact_inventory_availability fa
JOIN dim_film df ON fa.film_id = df.film_id
JOIN sakila.film f ON df.film_id = f.film_id  -- Join with the film table to get the title
GROUP BY f.title
ORDER BY total_inventory DESC
LIMIT 10;
""", con=sakila_project_engine)

print("Top 5 films by inventory count:")
inventory_summary.head(5)



Top 5 films by inventory count:


,title,total_inventory
0,ACADEMY DINOSAUR,1150.0
1,ADAPTATION HOLES,240.0
2,AFFAIR PREJUDICE,230.0
3,ACE GOLDFINGER,210.0
4,AFRICAN EGG,60.0


In [47]:
# Top 10 most rented films
top_rented_films_query = pd.read_sql("""
SELECT 
    f.title,  -- Fetch title from the film table
    COUNT(fr.rental_id) AS rental_count
FROM fact_rental fr
JOIN dim_film df ON fr.film_id = df.film_id
JOIN sakila.film f ON df.film_id = f.film_id  -- Join with film table to access title
GROUP BY f.title
ORDER BY rental_count DESC
LIMIT 10;
""", con=sakila_project_engine)

print("Top 10 Most Rented Films:")
top_rented_films_query.head(10)


Top 10 Most Rented Films:


,title,rental_count
0,BUCKET BROTHERHOOD,1156
1,ROCKETEER MOTHER,1089
2,RIDGEMONT SUBMARINE,1024
3,JUGGLER HARDLY,1024
4,SCALAWAG DUCK,1024
5,FORWARD TEMPLE,1024
6,GRIT CLOCKWORK,1024
7,WIFE TURN,961
8,ROBBERS JOON,961
9,APACHE DIVINE,961


In [48]:
# Monthly rental count
monthly_rentals_df = pd.read_sql("""
SELECT EXTRACT(YEAR_MONTH FROM rental_date) AS rental_month, COUNT(*) AS rental_count
FROM fact_rental
GROUP BY rental_month
ORDER BY rental_month;
""", con=sakila_project_engine)

print("Monthly Rental Counts:")
monthly_rentals_df.head(5)

Monthly Rental Counts:


,rental_month,rental_count
0,200505,1156
1,200506,2311
2,200507,6709
3,200508,5686
4,200602,182


In [49]:
# Top 10 customers by rental count
top_customers_df = pd.read_sql("""
SELECT dim_customer.customer_id, dim_customer.first_name, dim_customer.last_name, COUNT(fact_rental.rental_id) AS total_rentals
FROM fact_rental
JOIN dim_customer ON fact_rental.customer_id = dim_customer.customer_id
GROUP BY dim_customer.customer_id, dim_customer.first_name, dim_customer.last_name
ORDER BY total_rentals DESC
LIMIT 10;
""", con=sakila_project_engine)

print("Top 10 Customers by Rental Count:")
top_customers_df.head(10)


Top 10 Customers by Rental Count:


,customer_id,first_name,last_name,total_rentals
0,148,ELEANOR,HUNT,1012
1,526,KARL,SEAL,990
2,144,CLARA,SHAW,924
3,236,MARCIA,DEAN,924
4,75,TAMMY,SANDERS,902
5,469,WESLEY,BULL,880
6,197,SUE,PETERS,880
7,137,RHONDA,KENNEDY,858
8,178,MARION,SNYDER,858
9,468,TIM,CARY,858
